In [44]:
import os
import time
import requests
import json
import pandas as pd
import numpy as np
from pprint import pprint as pp
%matplotlib inline

In [47]:
## locations = ['The Villages', 'Lady Lake', 'Wildwood', 'Busnell', 'Center Hill','Webseter','Coleman', 'Leesburg', 'Fruitland Park', 'Mt. Dora', 'Belleville', 'Summerfield'] 

baseurl = "https://projects.propublica.org/nonprofits/api/v2/search.json?state%5Bid%5D=FL&page="
r = requests.get(baseurl)
rawdata = r.json()
nonprof = rawdata['organizations']
df = pd.DataFrame(nonprof)
for page in range(1, rawdata['num_pages']):
    print("Fetching " + str(page))
    r = requests.get(baseurl + str(page))
    rawdata = r.json()
    nonprof = rawdata['organizations']
    dfnew = pd.DataFrame(nonprof)
    df = df.append(dfnew)
    time.sleep(2)
df.shape
#request Florida nonprofit data from ProPublica's Nonprofit explorer API
# print(nonprof)
#Returns 43,941 Florida nonprofits/charities
#Narrow down to Lady Lake, Wildwood, The Villages, etc. 


Fetching 1
Fetching 2
Fetching 3
Fetching 4
Fetching 5
Fetching 6
Fetching 7
Fetching 8
Fetching 9
Fetching 10
Fetching 11
Fetching 12
Fetching 13
Fetching 14
Fetching 15
Fetching 16
Fetching 17
Fetching 18
Fetching 19
Fetching 20
Fetching 21
Fetching 22
Fetching 23
Fetching 24
Fetching 25
Fetching 26
Fetching 27
Fetching 28
Fetching 29
Fetching 30
Fetching 31
Fetching 32
Fetching 33
Fetching 34
Fetching 35
Fetching 36
Fetching 37
Fetching 38
Fetching 39
Fetching 40
Fetching 41
Fetching 42
Fetching 43
Fetching 44
Fetching 45
Fetching 46
Fetching 47
Fetching 48
Fetching 49
Fetching 50
Fetching 51
Fetching 52
Fetching 53
Fetching 54
Fetching 55
Fetching 56
Fetching 57
Fetching 58
Fetching 59
Fetching 60
Fetching 61
Fetching 62
Fetching 63
Fetching 64
Fetching 65
Fetching 66
Fetching 67
Fetching 68
Fetching 69
Fetching 70
Fetching 71
Fetching 72
Fetching 73
Fetching 74
Fetching 75
Fetching 76
Fetching 77
Fetching 78
Fetching 79
Fetching 80
Fetching 81
Fetching 82
Fetching 83
Fetching 84
F

(43941, 14)

In [48]:
print(r.content)

b'{"total_results":43941,"organizations":[{"ein":391097290,"strein":"39-1097290","name":"YOUNGER FAMILY FOUNDATION INC","sub_name":"YOUNGER FAMILY FOUNDATION INC","city":"NAPLES","state":"FL","ntee_code":"T20","raw_ntee_code":"T20","subseccd":3,"has_subseccd":true,"have_filings":true,"have_extracts":true,"have_pdfs":true,"score":4.271313},{"ein":592432681,"strein":"59-2432681","name":"YOUTH AUTOMOTIVE TRAINING CENTER OF BROWARD INC","sub_name":"YOUTH AUTOMOTIVE TRAINING CENTER OF BROWARD INC","city":"DEERFIELD BCH","state":"FL","ntee_code":null,"raw_ntee_code":null,"subseccd":3,"has_subseccd":true,"have_filings":true,"have_extracts":true,"have_pdfs":true,"score":4.271313},{"ein":454546758,"strein":"45-4546758","name":"YOUTH COMBINE INC","sub_name":"YOUTH COMBINE INC","city":"GAINESVILLE","state":"FL","ntee_code":"N99","raw_ntee_code":"N99","subseccd":3,"has_subseccd":true,"have_filings":true,"have_extracts":true,"have_pdfs":true,"score":4.271313},{"ein":461066964,"strein":"46-1066964",

In [49]:
df.head()

,city,ein,has_subseccd,have_extracts,have_filings,have_pdfs,name,ntee_code,raw_ntee_code,score,state,strein,sub_name,subseccd
0,HURLBURT FLD,562342625,True,False,True,True,1 SOSFS DEFENDERS ASSOCIATION,None,None,4.283534,FL,56-2342625,1 SOSFS DEFENDERS ASSOCIATION,3
1,PENSACOLA,593068740,True,False,True,True,100 BLACK MEN OF PENSACOLA INC,S82Z,S82Z,4.283534,FL,59-3068740,100 BLACK MEN OF PENSACOLA INC,3
2,DEERFIELD BCH,591756721,True,True,True,True,100 CLUB OF SOUTH PALM BEACH COUNTY INC,None,None,4.283534,FL,59-1756721,100 CLUB OF SOUTH PALM BEACH COUNTY INC,3
3,PONTE VEDRA,261211203,True,True,True,True,100 TIMES FOUNDATION CORPORATION,T20,T20,4.283534,FL,26-1211203,100 TIMES FOUNDATION CORPORATION,3
4,FORT LAUDERDALE,800364095,True,True,True,True,1000 PLUS CLUB TO BENEFIT CANCER INC,G113,G113,4.283534,FL,80-0364095,1000 PLUS CLUB TO BENEFIT CANCER INC,3


In [52]:
with open("data/florida_nonprofs.json", "w") as file:
    file.write(json.dumps(rawdata))

S/O to Mike Stucka (@MikeStucka) for helping me figure out how to properly parse ProPublica's Nonprofit API